In [ ]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
#import pandas_ta as ta
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

#pip install statsmodels pandas-datareader matplotlib pandas numpy datetime yfinance pandas_ta

Traceback (most recent call last):
  File "c:\Users\William\.vscode\extensions\ms-python.python-2024.22.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 9, in <module>
  File "G:\Min enhet\Industriell Ekonomi\Årskurs 4\AlgTrading\Algorithmic_Trading\.venv\Lib\site-packages\pandas_ta\__init__.py", line 116, in <module>
    from pandas_ta.core import *
  File "G:\Min enhet\Industriell Ekonomi\Årskurs 4\AlgTrading\Algorithmic_Trading\.venv\Lib\site-packages\pandas_ta\core.py", line 18, in <module>
    from pandas_ta.momentum import *
  File "G:\Min enhet\Industriell Ekonomi\Årskurs 4\AlgTrading\Algorithmic_Trading\.venv\Lib\site-packages\pandas_ta\momentum\__init__.py", line 34, in <module>
    from .squeeze_pro import squeeze_pro
  File "G:\Min enhet\Industriell Ekonomi\Årskurs 4\AlgTrading\Algorithmic_Trading\.venv\Lib\site-packages\pandas_ta\

In [ ]:
# Set up dataframe with S&P500 stocks
SP500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

SP500['Symbol'] = SP500['Symbol'].str.replace('.', '-')
symbols_list = SP500['Symbol'].unique().tolist()

end_date = '2024-12-01'
start_date = pd.to_datetime(end_date) - pd.DateOffset(365 * 8)  # Eight years of time

df = yf.download(tickers=symbols_list, 
                 start=start_date,
                 end=end_date).stack()
# Stack makes sure each row is unique, pivots columns into a multi-level index

df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()  # Convert all column names to lower case

# Uncomment the line below if you want to see the dataframe
# print(df)


In [ ]:
# Garman-Klass volatility, estimates volatility using FOUR key metrics
# High: Highest price of asset during period
# Low: Lowest price of asset during period
# Close: Closing price of asset
# Open: Opening price of asset
df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low'])) ** 2) / 2 - ((2 * np.log(2) - 1) * (np.log(df['adj close']) - np.log(df['open'])) ** 2)
print(df)

#Relative Strength Index (RSI), used to measure speed and change of price movements of a stock from 0 to 100

#df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: ta.rsi(close=x, length=20))

Price               adj close       close  ...       volume  garman_klass_vol
date       ticker                          ...                               
2016-12-05 A        41.855900   44.529999  ...    2495000.0         -0.001098
           AAPL     25.262007   27.277500  ...  137298000.0         -0.002650
           ABBV     43.165909   60.860001  ...   10227700.0         -0.039830
           ABT      33.251614   38.430000  ...    9514700.0         -0.007112
           ACGL     26.609310   27.983334  ...    1685400.0         -0.000483
...                       ...         ...  ...          ...               ...
2024-11-29 XYL     126.750000  126.750000  ...    1084000.0          0.000093
           YUM     138.270004  138.940002  ...    1003200.0          0.000017
           ZBH     112.099998  112.099998  ...     504100.0          0.000039
           ZBRA    407.000000  407.000000  ...     175700.0          0.000037
           ZTS     175.250000  175.250000  ...    1543400.0     